In [250]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import patches
%matplotlib inline
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fonm
from matplotlib import font_manager, rc
mpl.rcParams['axes.unicode_minus'] = False
font_path = "C:/Windows/Fonts/HMKMRHD.ttf"
font = fonm.FontProperties(fname=font_path).get_name()
rc('font', family=font)

In [251]:
Basic_dir = os.path.dirname(os.path.dirname(os.path.realpath('EDA.ipynb')))
PATH = Basic_dir + '\\Data\\Data(PenW_B)'
os.listdir(PATH)

['sample_submission(0).csv',
 'sample_submission(1).csv',
 'sample_submission(2).csv',
 'sample_submission(3).csv',
 'sample_submission(3_statsmodel).csv',
 'sample_submission.csv',
 'test.csv',
 'train.csv',
 'train_Ch.csv',
 'train_pred_sex.csv']

In [252]:
train = pd.read_csv(PATH + '\\train.csv')
# train.drop(['Body Mass (g)'], axis = 1, inplace = True)
train.head()

,id,Species,Island,Clutch Completion,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo),Body Mass (g)
0,0,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,50.0,15.3,220,MALE,8.30515,-25.19017,5550
1,1,Chinstrap penguin (Pygoscelis antarctica),Dream,No,49.5,19.0,200,MALE,9.63074,-24.34684,3800
2,2,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,45.1,14.4,210,FEMALE,8.51951,-27.01854,4400
3,3,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,44.5,14.7,214,FEMALE,8.20106,-26.16524,4850
4,4,Gentoo penguin (Pygoscelis papua),Biscoe,No,49.6,16.0,225,MALE,8.38324,-26.84272,5700


In [253]:
# 결측값이 있는 행
for i in range(len((train.isna().sum(axis=1)>=1))):
    if (train.isna().sum(axis=1)>=1)[i] == True:
        print(i)

6
8
18
70
109


In [254]:
# 누락값 상세 확인
null_list = [6, 8, 18, 70, 109]
train_null = train.iloc[null_list, :].reset_index(drop=True)

이전 분석(EDA)에서 Species, Island, Clutch Completion은 Sex와 연관성이 없는 것으로 확인
연속형 변수들과의 연관성 확인 필요

In [255]:
# na 값 삭제
train.dropna(inplace=True)
train.reset_index(drop=True, inplace=True)

In [256]:
# 데이터 분석을 위한 원-핫 인코딩 
sp_list = list(train.Species.unique())
sp_index = [x for x in range(len(sp_list))]
sp_dict = dict(zip(sp_list, sp_index)) 
train.Species = train.Species.apply(lambda x: sp_dict[x])

land_list = list(train.Island.unique())
land_index = [x for x in range(len(land_list))]
land_dict = dict(zip(land_list, land_index)) 
train.Island = train.Island.apply(lambda x: land_dict[x])

CC_list = list(train['Clutch Completion'].unique())
CC_index = [x for x in range(len(CC_list))]
CC_dict = dict(zip(CC_list, CC_index)) 
train['Clutch Completion'] = train['Clutch Completion'].apply(lambda x: CC_dict[x])

sex_list = list(train.Sex.unique())
sex_index = [x for x in range(len(sex_list))]
sex_dict = dict(zip(sex_list, sex_index)) 
train.Sex = train.Sex.apply(lambda x: sex_dict[x])

train.drop(['id'], axis=1, inplace=True)
train.dtypes

Species                  int64
Island                   int64
Clutch Completion        int64
Culmen Length (mm)     float64
Culmen Depth (mm)      float64
Flipper Length (mm)      int64
Sex                      int64
Delta 15 N (o/oo)      float64
Delta 13 C (o/oo)      float64
Body Mass (g)            int64
dtype: object

In [257]:
train.columns = ['Species', 'Island', 'Clutch Completion', 'Culmen_Length',
                 'Culmen_Depth', 'Flipper_Length', 'Sex', 'Delta_15_N',
                 'Delta_13_C', 'Body_Mass']

In [258]:
# 크루스칼 왈리스 검정 실시 # 등분산성 및 정규성 검정은 미실시
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

print('Culmen_Length:', stats.kruskal(train[train.Sex == 0]['Culmen_Length'],\
              train[train.Sex == 1]['Culmen_Length']))

print('Delta_15_N:', stats.kruskal(train[train.Sex == 0]['Delta_15_N'],\
              train[train.Sex == 1]['Delta_15_N']))    

print('Delta_13_C:', stats.kruskal(train[train.Sex == 0]['Delta_13_C'],\
              train[train.Sex == 1]['Delta_13_C']))     

print('Culmen_Depth:', stats.kruskal(train[train.Sex == 0]['Culmen_Depth'],\
              train[train.Sex == 1]['Culmen_Depth']))       

print('Flipper_Length:', stats.kruskal(train[train.Sex == 0]['Culmen_Depth'],\
              train[train.Sex == 1]['Flipper_Length']))               

# 성별과 Culmen_Depth, Culmen_Length, Flipper_Length는 연관성이 있음.

Culmen_Length: KruskalResult(statistic=12.188309936945336, pvalue=0.00048089932984587573)
Delta_15_N: KruskalResult(statistic=1.9430670339760923, pvalue=0.16333590696422495)
Delta_13_C: KruskalResult(statistic=1.1377777777777283, pvalue=0.28612238439102555)
Culmen_Depth: KruskalResult(statistic=14.545073083944217, pvalue=0.0001368460352194139)
Flipper_Length: KruskalResult(statistic=81.04092976250625, pvalue=2.2109079001869885e-19)


In [259]:
# 예측모델 작성 - 로지스틱 회귀
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
val_list = ['Culmen_Depth', 'Culmen_Length', 'Flipper_Length']
X_train, X_test, y_train, y_test = train_test_split(train[val_list], train[['Sex']], test_size = 0.2, random_state = 7)
lr = LogisticRegression()
lr.fit(X_train, list(y_train.Sex))
# 성능평가(훈련)
print(classification_report(y_train, lr.predict(X_train)))
# 성능평가(훈련)
print(classification_report(y_test, lr.predict(X_test)))

              precision    recall  f1-score   support

           0       0.85      0.76      0.81        38
           1       0.83      0.90      0.86        49

    accuracy                           0.84        87
   macro avg       0.84      0.83      0.83        87
weighted avg       0.84      0.84      0.84        87

              precision    recall  f1-score   support

           0       0.93      0.76      0.84        17
           1       0.50      0.80      0.62         5

    accuracy                           0.77        22
   macro avg       0.71      0.78      0.73        22
weighted avg       0.83      0.77      0.79        22



In [260]:
# 성능평가(훈련)
print(confusion_matrix(y_test, lr.predict(X_test)))
print(classification_report(y_test, lr.predict(X_test)))

[[13  4]
 [ 1  4]]
              precision    recall  f1-score   support

           0       0.93      0.76      0.84        17
           1       0.50      0.80      0.62         5

    accuracy                           0.77        22
   macro avg       0.71      0.78      0.73        22
weighted avg       0.83      0.77      0.79        22



In [261]:
# KNN 모델
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
val_list = ['Culmen_Depth', 'Culmen_Length', 'Flipper_Length']
X_train, X_test, y_train, y_test = train_test_split(train[val_list], train[['Sex']], test_size = 0.2, random_state = 7)
from sklearn.metrics import accuracy_score
n = 7
neigh_n = KNeighborsClassifier(n_neighbors = n, weights = 'distance')
neigh_n.fit(X_train, list(y_train.Sex))
y_pred = neigh_n.predict(X_train)
accuracy = accuracy_score(y_train, y_pred)
print(classification_report(y_train, y_pred))
print(f'Accuracy : {accuracy*100}')

y_pred = neigh_n.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(classification_report(y_test, y_pred))
print(f'Accuracy : {accuracy*100}')

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        38
           1       1.00      1.00      1.00        49

    accuracy                           1.00        87
   macro avg       1.00      1.00      1.00        87
weighted avg       1.00      1.00      1.00        87

Accuracy : 100.0
              precision    recall  f1-score   support

           0       0.94      0.88      0.91        17
           1       0.67      0.80      0.73         5

    accuracy                           0.86        22
   macro avg       0.80      0.84      0.82        22
weighted avg       0.88      0.86      0.87        22

Accuracy : 86.36363636363636


In [262]:
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train[val_list], train[['Sex']], test_size = 0.2, random_state = 7)
model = DecisionTreeClassifier()
model.fit(X_train, list(y_train.Sex))
# 학습데이터에 대한 성능평가
# print(confusion_matrix(y_train, model.predict(X_train)))
print(classification_report(y_train, model.predict(X_train)))
# 훈련데이터에 대한 성능평가
# print(confusion_matrix(y_train, model.predict(X_train)))
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        38
           1       1.00      1.00      1.00        49

    accuracy                           1.00        87
   macro avg       1.00      1.00      1.00        87
weighted avg       1.00      1.00      1.00        87

              precision    recall  f1-score   support

           0       0.93      0.76      0.84        17
           1       0.50      0.80      0.62         5

    accuracy                           0.77        22
   macro avg       0.71      0.78      0.73        22
weighted avg       0.83      0.77      0.79        22



In [263]:
train_null.columns = ['id', 'Species', 'Island', 'Clutch Completion', 'Culmen_Length',
                      'Culmen_Depth', 'Flipper_Length', 'Sex', 'Delta_15_N',
                      'Delta_13_C', 'Body_Mass']
train_null
lr.predict(train_null[val_list]), neigh_n.predict(train_null[val_list]), model.predict(train_null[val_list])

(array([0, 1, 1, 1, 1]), array([0, 1, 1, 1, 1]), array([0, 1, 0, 1, 1]))

In [264]:
pred_sex = neigh_n.predict(train_null[val_list])
for i in range(len(pred_sex)):
    train_null.Sex[i] = pred_sex[i]

# 데이터 분석을 위한 원-핫 인코딩 
sp_list = list(train_null.Species.unique())
sp_index = [x for x in range(len(sp_list))]
sp_dict = dict(zip(sp_list, sp_index)) 
train_null.Species = train_null.Species.apply(lambda x: sp_dict[x])

land_list = list(train_null.Island.unique())
land_index = [x for x in range(len(land_list))]
land_dict = dict(zip(land_list, land_index)) 
train_null.Island = train_null.Island.apply(lambda x: land_dict[x])

CC_list = list(train_null['Clutch Completion'].unique())
CC_index = [x for x in range(len(CC_list))]
CC_dict = dict(zip(CC_list, CC_index)) 
train_null['Clutch Completion'] = train_null['Clutch Completion'].apply(lambda x: CC_dict[x])

train_null.drop(['id'], axis=1, inplace=True)
train.dtypes
train_null

<ipython-input-264-6b8f09619b22>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_null.Sex[i] = pred_sex[i]


,Species,Island,Clutch Completion,Culmen_Length,Culmen_Depth,Flipper_Length,Sex,Delta_15_N,Delta_13_C,Body_Mass
0,0,0,0,42.0,20.2,190,0,9.13362,-25.09368,4250
1,0,0,0,34.1,18.1,193,1,NaN,NaN,3475
2,0,1,1,39.8,19.1,184,1,NaN,NaN,4650
3,1,2,0,46.2,14.4,214,1,8.24253,-26.81540,4650
4,0,0,0,36.6,17.8,185,1,NaN,NaN,3700


In [265]:
train_pred_sex = pd.concat([train, train_null]).reset_index(drop=True)
train_pred_sex.to_csv(PATH + '\\train_pred_sex.csv', index = False)
pd.read_csv(PATH + '\\train_pred_sex.csv').head()

,Species,Island,Clutch Completion,Culmen_Length,Culmen_Depth,Flipper_Length,Sex,Delta_15_N,Delta_13_C,Body_Mass
0,0,0,0,50.0,15.3,220,0,8.30515,-25.19017,5550
1,1,1,1,49.5,19.0,200,0,9.63074,-24.34684,3800
2,0,0,0,45.1,14.4,210,1,8.51951,-27.01854,4400
3,0,0,0,44.5,14.7,214,1,8.20106,-26.16524,4850
4,0,0,1,49.6,16.0,225,0,8.38324,-26.84272,5700


In [266]:
print(train[train.Species == 0].Body_Mass.mean(), train[train.Species == 1].Body_Mass.mean(), train[train.Species == 2].Body_Mass.mean())
print(train[train.Species == 0].Delta_15_N.mean(), train[train.Species == 1].Delta_15_N.mean(), train[train.Species == 2].Delta_15_N.mean())
print(train[train.Species == 0].Delta_13_C.mean(), train[train.Species == 1].Delta_13_C.mean(), train[train.Species == 2].Delta_13_C.mean())
Speices0_D15N = train[train.Species == 0].Delta_15_N.mean()
Speices0_D13C = train[train.Species == 0].Delta_13_C.mean()

5084.574468085107 3756.0 3777.7027027027025
8.277214680851063 9.4173136 8.865927297297297
-26.257984468085105 -24.515759600000003 -25.84676567567568


In [ ]:
train = pd.read_csv(PATH + '\\train_pred_sex.csv')
# 결측값이 있는 행
null_list = []
for i in range(len((train.isna().sum(axis=1)>=1))):
    if (train.isna().sum(axis=1)>=1)[i] == True:
        null_list.append(i)
for i in null_list:
    train.Delta_15_N[i] = Speices0_D15N
    train.Delta_13_C[i] = Speices0_D13C
train.isna().sum()

In [268]:
train.to_csv(PATH + '\\train_Ch.csv', index = False)
pd.read_csv(PATH + '\\train_Ch.csv').isna().sum()

Species              0
Island               0
Clutch Completion    0
Culmen_Length        0
Culmen_Depth         0
Flipper_Length       0
Sex                  0
Delta_15_N           0
Delta_13_C           0
Body_Mass            0
dtype: int64

In [282]:
test = pd.read_csv(PATH + '\\test.csv')
test.Sex = test.Sex.apply(lambda x: str(x))
null_id = []
for i in range(len(test)):
    if test.Sex[i] == 'nan':
        null_id.append(test.id[i])
test.iloc[null_id, :]
test.columns = ['id', 'Species', 'Island', 'Clutch Completion', 'Culmen_Length',
                      'Culmen_Depth', 'Flipper_Length', 'Sex', 'Delta_15_N',
                      'Delta_13_C']

# 데이터 분석을 위한 원-핫 인코딩 
sp_list = list(test.Species.unique())
sp_index = [x for x in range(len(sp_list))]
sp_dict = dict(zip(sp_list, sp_index)) 
test.Species = test.Species.apply(lambda x: sp_dict[x])

land_list = list(test.Island.unique())
land_index = [x for x in range(len(land_list))]
land_dict = dict(zip(land_list, land_index)) 
test.Island = test.Island.apply(lambda x: land_dict[x])

CC_list = list(test['Clutch Completion'].unique())
CC_index = [x for x in range(len(CC_list))]
CC_dict = dict(zip(CC_list, CC_index)) 
test['Clutch Completion'] = test['Clutch Completion'].apply(lambda x: CC_dict[x])

sex_list = list(test.Sex.unique())
sex_index = [x for x in range(len(sex_list))]
sex_dict = dict(zip(sex_list, sex_index)) 
test.Sex = test.Sex.apply(lambda x: sex_dict[x])

print(test[test.Species == 0].Delta_15_N.mean(), test[test.Species == 1].Delta_15_N.mean(), test[test.Species == 2].Delta_15_N.mean())
print(test[test.Species == 0].Delta_13_C.mean(), test[test.Species == 1].Delta_13_C.mean(), test[test.Species == 2].Delta_13_C.mean())
pred = neigh_n.predict(test.iloc[null_id,:][val_list])
for i in null_id:
    k = 0
    test.Sex[i] = pred[k]
    k += 1
test.iloc[null_id,:]

9.319750714285718 8.225130405405409 8.854848446601938
-24.5644388372093 -26.130616891891886 -25.795799126213588


<ipython-input-282-aa6800c6d419>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.Sex[i] = pred[k]


,id,Species,Island,Clutch Completion,Culmen_Length,Culmen_Depth,Flipper_Length,Sex,Delta_15_N,Delta_13_C
46,46,2,2,0,37.8,17.1,186.0,1,8.63243,-25.21315
81,81,2,0,0,37.5,18.9,179.0,1,NaN,NaN
98,98,1,1,0,47.3,13.8,216.0,1,8.25818,-26.23886
152,152,1,1,0,44.5,15.7,217.0,1,8.04111,-26.18444
205,205,2,2,0,37.8,17.3,180.0,1,NaN,NaN
209,209,1,1,0,44.5,14.3,216.0,1,7.96621,-25.69327


In [306]:
test['Delta_15_N'].fillna(test['Delta_15_N'].mean(), inplace=True)
test['Delta_13_C'].fillna(test['Delta_13_C'].mean(), inplace=True)

In [307]:
test.isna().sum()

id                   0
Species              0
Island               0
Clutch Completion    0
Culmen_Length        0
Culmen_Depth         0
Flipper_Length       0
Sex                  0
Delta_15_N           0
Delta_13_C           0
dtype: int64

In [308]:
test.to_csv(PATH + '\\test_Ch.csv', index = False)
pd.read_csv(PATH + '\\test_Ch.csv').isna().sum()

id                   0
Species              0
Island               0
Clutch Completion    0
Culmen_Length        0
Culmen_Depth         0
Flipper_Length       0
Sex                  0
Delta_15_N           0
Delta_13_C           0
dtype: int64